In [37]:
from datetime import datetime, timedelta
import pytz
import pandas as pd
from tqdm import tqdm
from modeler.modeler import Modeler as m
from processor.processor import Processor as p
from database.market import Market
from database.sec import SEC
from database.adatabase import ADatabase
import numpy as np
import matplotlib.pyplot as plt
import pickle
from functional.backtester import Backtester as b
from statistics import variance
import math

In [38]:
start_date = datetime(2020,1,1)
end_date = datetime(2023,1,1)

In [39]:
market = Market()
fin_db = ADatabase("dividends")
speculation_db = ADatabase("spec")
classification_db = ADatabase("spec_classification")
experimental = ADatabase("dividends_spec")
sec = SEC()

In [40]:
market.connect()
sp500 = market.retrieve("sp500")
market.disconnect()
sp500 = sp500.rename(columns={"Symbol":"ticker"})

In [41]:
bench = pd.read_csv("./csv_files/FED/SPY.csv")
bench = p.column_date_processing(bench)
bench["day"] = [x.weekday() for x in bench["date"]]
bench_returns = bench.copy()
bench_returns[f"bench_return"] = (bench_returns["adjclose"].shift(-1) - bench_returns["adjclose"].shift(4)) / bench_returns["adjclose"].shift(4)
bench_quarterlies = bench_returns.groupby(["year","quarter"]).agg({"adjclose":"first"}).reset_index().rename(columns={"adjclose":"quarter_start"})
bench_returns = bench_returns.merge(bench_quarterlies,on=["year","quarter"])
bench_returns[f"bench_quarterly_return"] = (bench_returns["adjclose"].shift(-1) - bench_returns["quarter_start"]) / bench_returns["quarter_start"]
bench_returns["variance"] = bench_returns["bench_return"].rolling(window=14).var()
bench_returns["quarterly_variance"] = bench_returns["bench_quarterly_return"].rolling(window=14).var()
bench_returns = bench_returns.dropna()

In [42]:
bench_returns

,date,open,high,low,close,adjclose,volume,year,quarter,week,day,bench_return,quarter_start,bench_quarterly_return,variance,quarterly_variance
17,1993-05-24,44.843750,45.656250,44.843750,45.218750,25.977037,457600,1993,2,21,0,0.028389,25.438465,0.022583,0.000334,0.000168
18,1993-05-31,45.375000,45.812500,45.093750,45.281250,26.012943,102200,1993,2,22,0,0.016914,25.438465,0.018349,0.000331,0.000168
19,1993-06-07,45.375000,45.375000,44.718750,45.093750,25.905237,934500,1993,2,23,0,0.011364,25.438465,0.004940,0.000333,0.000153
20,1993-06-14,45.156250,45.187500,44.500000,44.500000,25.564137,633400,1993,2,24,0,0.007792,25.438465,0.018460,0.000333,0.000141
21,1993-06-21,44.625000,44.906250,44.218750,44.781250,25.908054,682900,1993,2,25,0,-0.004745,25.438465,0.016326,0.000262,0.000121
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1577,2023-04-17,412.369995,415.720001,410.170013,412.200012,412.200012,334521500,2023,2,16,0,0.050992,409.190002,0.016472,0.001976,0.000543
1578,2023-04-24,411.989990,415.940002,403.779999,415.929993,415.929993,424849800,2023,2,17,0,0.007914,409.190002,0.008407,0.001872,0.000563
1579,2023-05-01,415.470001,417.619995,403.739990,412.630005,412.630005,440398500,2023,2,18,0,0.005865,409.190002,0.005865,0.001584,0.000500
1580,2023-05-08,412.970001,414.540009,408.869995,411.589996,411.589996,336006300,2023,2,19,0,0.014935,409.190002,0.023046,0.001466,0.000443


In [43]:
tyields = pd.read_csv("./csv_files/FED/DGS1.csv")
tyields = p.column_date_processing(tyields)
tyields["dgs1"] = tyields["dgs1"].replace(".",0)
tyields["dgs1"] = tyields["dgs1"].astype("float")
tyields["yield"] = [1+(x/100) for x in tyields["dgs1"]]
tyields["weekly_yield"] = [math.exp(math.log(x)/52) for x in tyields["yield"]]
tyields["quarterly_yield"] = [math.exp(math.log(x)/4) for x in tyields["yield"]]
tyields = tyields.dropna()

In [44]:
tyields

,date,dgs1,year,quarter,week,yield,weekly_yield,quarterly_yield
0,1962-01-02,3.22,1962,1,1,1.0322,1.000610,1.007955
1,1962-01-03,3.24,1962,1,1,1.0324,1.000613,1.008003
2,1962-01-04,3.24,1962,1,1,1.0324,1.000613,1.008003
3,1962-01-05,3.26,1962,1,1,1.0326,1.000617,1.008052
4,1962-01-08,3.31,1962,1,2,1.0331,1.000626,1.008174
...,...,...,...,...,...,...,...,...
16008,2023-05-12,4.75,2023,2,19,1.0475,1.000893,1.011669
16009,2023-05-15,4.73,2023,2,20,1.0473,1.000889,1.011621
16010,2023-05-16,4.88,2023,2,20,1.0488,1.000917,1.011983
16011,2023-05-17,4.92,2023,2,20,1.0492,1.000924,1.012079


In [45]:
new_prices = []
market.connect()
sec.connect()
for ticker in tqdm(sp500["ticker"].unique()):
    try:
        cik = int(sp500[sp500["ticker"]==ticker]["CIK"])
        financials = sec.retrieve_filing_data(cik)
        financials = p.column_date_processing(financials)
        ticker_sim = market.retrieve_ticker_prices("prices",ticker)
        ticker_sim = p.column_date_processing(ticker_sim)
        ticker_sim = ticker_sim[(ticker_sim["date"]>=(start_date-timedelta(days=365)))]
        for i in range(2,5):
            ticker_sim[f"return_{i}"] = (ticker_sim["adjclose"].shift(-i) - ticker_sim["adjclose"].shift(-1)) / ticker_sim["adjclose"].shift(-1)
        quarterlies = ticker_sim.groupby(["year","quarter"]).agg({"adjclose":"first"}).reset_index().rename(columns={"adjclose":"quarter_start"})
        if "commonstockdividendspersharecashpaid" in financials.columns:
            quarterlies = quarterlies.merge(financials[["year","quarter","commonstockdividendspersharecashpaid"]],how="left")
            quarterlies["commonstockdividendspersharecashpaid"] = quarterlies["commonstockdividendspersharecashpaid"].fillna(0)
        else:
            quarterlies["commonstockdividendspersharecashpaid"] = 0
        ticker_sim = ticker_sim.merge(quarterlies,on=["year","quarter"],how="left")
        ticker_sim["quarterly_return"] =  (ticker_sim["adjclose"].shift(-1) - ticker_sim["quarter_start"]) / ticker_sim["quarter_start"]
        ticker_sim["day"] = [x.weekday() for x in ticker_sim["date"]]
        returns = ticker_sim[ticker_sim["day"]==0]
        returns["weekly_return"] = [row[1]["return_4"] if row[1]["week"] % (13) != 2 else row[1]["return_4"] + row[1]["commonstockdividendspersharecashpaid"] for row in returns.iterrows()]
        new_sim = ticker_sim.merge(returns[["year","week","weekly_return"]], on=["year","week"],how="left") \
                            .merge(bench_returns[["year","week","bench_return","variance","bench_quarterly_return","quarterly_variance"]],on=["year","week"],how="left")
        new_sim = new_sim.dropna()
        new_sim["market_cov"] = new_sim["weekly_return"].rolling(window=100).cov(new_sim["bench_return"])
        new_sim["market_quarterly_cov"] = new_sim["quarterly_return"].rolling(window=100).cov(new_sim["bench_quarterly_return"])
        completed = new_sim.copy()
        completed["beta"] = completed["market_cov"] / completed["variance"]
        completed["quarterly_beta"] = completed["market_quarterly_cov"] / completed["quarterly_variance"]
        completed  = completed.dropna()
        new_prices.append(completed)
    except Exception as e:
        print(str(e))
        continue
sec.disconnect()
market.disconnect()
price_returns = pd.concat(new_prices)

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 490/490 [03:03<00:00,  2.67it/s]


In [46]:
returns

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,...,quarter,week,return_2,return_3,return_4,quarter_start,commonstockdividendspersharecashpaid,quarterly_return,day,weekly_return
3,2019-01-07,85.27,86.57,84.8300,84.83,2360785,82.813042,84.075584,82.385720,82.385720,...,1,2,-0.003582,-0.007395,-0.005547,82.045805,0.457667,0.024384,0,0.452120
8,2019-01-14,84.50,85.70,84.4100,85.34,1592718,82.065229,83.230652,81.977822,82.881025,...,1,3,-0.018866,-0.004669,-0.000652,82.045805,0.457667,0.003906,0,-0.000652
17,2019-01-28,83.65,83.96,82.9600,83.92,2471773,81.397861,81.699514,80.726438,81.660591,...,1,5,0.008056,0.020732,0.032342,82.045805,0.457667,0.001116,0,0.032342
22,2019-02-04,86.97,87.38,86.6100,87.25,1976959,84.628475,85.027436,84.278167,84.900936,...,1,6,0.002404,-0.013277,-0.006181,82.045805,0.457667,0.036222,0,-0.006181
27,2019-02-11,86.27,87.20,85.8600,86.91,3361139,83.947321,84.852283,83.548360,84.570090,...,1,7,0.008456,0.065707,0.089818,82.045805,0.457667,0.037883,0,0.089818
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1205,2023-04-17,174.62,174.81,172.3122,174.01,1284951,174.245333,174.434925,171.942484,173.636641,...,2,16,0.003038,0.001863,0.016110,167.300266,0.000000,0.040499,0,0.016110
1210,2023-04-24,176.35,177.44,175.7200,176.99,1117746,176.350000,177.440000,175.720000,176.990000,...,2,17,-0.000462,0.005375,0.015952,167.300266,0.000000,0.034188,0,0.015952
1215,2023-05-01,179.56,179.94,175.5400,175.72,1689966,179.560000,179.940000,175.540000,175.720000,...,2,18,0.004692,0.008875,0.052742,167.300266,0.000000,0.057380,0,0.052742
1220,2023-05-08,184.87,187.05,183.5000,186.31,1976167,184.870000,187.050000,183.500000,186.310000,...,2,19,0.004176,0.004068,-0.004068,167.300266,0.000000,0.102031,0,-0.004068


In [47]:
price_returns

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,...,day,weekly_return,bench_return,variance,bench_quarterly_return,quarterly_variance,market_cov,market_quarterly_cov,beta,quarterly_beta
111,2019-06-12,169.01,170.1500,168.59,169.87,2166416,146.741640,147.731436,146.376978,147.488328,...,2,-0.013143,0.028547,0.001158,0.018817,0.002163,0.000292,0.003979,0.252232,1.840029
112,2019-06-13,168.85,169.9000,167.87,169.36,2141467,146.602721,147.514375,145.751843,147.045525,...,3,-0.013143,0.028547,0.001158,0.018817,0.002163,0.000290,0.004024,0.250460,1.860928
113,2019-06-14,166.69,168.2500,166.05,168.24,2719182,144.727317,146.081776,144.171642,146.073093,...,4,-0.013143,0.028547,0.001158,0.018817,0.002163,0.000288,0.004039,0.248640,1.867540
114,2019-06-17,166.78,167.4300,166.46,167.10,1783368,144.805459,145.369817,144.527622,145.083297,...,0,0.008670,0.041181,0.001221,0.020290,0.001839,0.000288,0.004045,0.235480,2.199842
115,2019-06-18,171.86,172.2700,167.57,167.85,3623040,149.216130,149.572110,145.491371,145.734479,...,1,0.008670,0.041181,0.001221,0.020290,0.001839,0.000289,0.004069,0.236991,2.212961
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1445,2023-05-09,184.37,185.2999,183.37,183.64,1904168,184.370000,185.299900,183.370000,183.640000,...,1,-0.004068,0.014935,0.001466,0.023046,0.000443,0.000330,0.000388,0.224855,0.876947
1446,2023-05-10,185.14,186.4900,183.35,186.00,1610464,185.140000,186.490000,183.350000,186.000000,...,2,-0.004068,0.014935,0.001466,0.023046,0.000443,0.000311,0.000401,0.212266,0.905065
1447,2023-05-11,185.12,186.2750,183.20,185.84,1539052,185.120000,186.275000,183.200000,185.840000,...,3,-0.004068,0.014935,0.001466,0.023046,0.000443,0.000292,0.000414,0.199442,0.935372
1448,2023-05-12,183.62,184.6300,181.79,184.38,1267674,183.620000,184.630000,181.790000,184.380000,...,4,-0.004068,0.014935,0.001466,0.023046,0.000443,0.000273,0.000427,0.186382,0.965836


In [48]:
speculation_db.connect()
simulation = speculation_db.retrieve("sim")
speculation_db.disconnect()

In [49]:
fin_db.connect()
financial_simulation = fin_db.retrieve("sim")
fin_db.disconnect()
financial_simulation.rename(columns={"prediction":"quarterly_prediction"},inplace=True)
financial_simulation["year"] = financial_simulation["year"] + 1

In [50]:
classification_db.connect()
classification_simulation = classification_db.retrieve("sim")
classification_db.disconnect()

In [51]:
simulation["week"] = simulation["week"] + 1
classification_simulation["week"] = classification_simulation["week"] + 1
classification_simulation.rename(columns={"prediction":"classification_prediction"},inplace=True)

In [52]:
simulation = price_returns.merge(simulation.drop("adjclose",axis=1)[["year","week","ticker","prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(classification_simulation.drop(["adjclose","training_years"],axis=1)[["year","week","ticker","classification_prediction"]],on=["year","week","ticker"],how="left") \
                    .merge(tyields[["date","weekly_yield","quarterly_yield"]],on="date",how="left") \
                    .merge(financial_simulation[["year","quarter","ticker","quarterly_prediction"]],on=["year","quarter","ticker"],how="left")

In [53]:
sim = simulation.dropna()
sim

,date,close,high,low,open,volume,adjclose,adjhigh,adjlow,adjopen,...,quarterly_variance,market_cov,market_quarterly_cov,beta,quarterly_beta,prediction,classification_prediction,weekly_yield,quarterly_yield,quarterly_prediction
3319,2020-01-06,89.40,89.6200,87.98,88.22,6782438,76.417295,76.605346,75.203508,75.408655,...,0.000953,0.000092,0.004903,0.373763,5.145854,78.469070,False,1.000294,1.003828,118.248221
3320,2020-01-07,88.89,89.5400,88.26,88.93,7242787,75.981357,76.536964,75.442846,76.015548,...,0.000953,0.000091,0.004795,0.371761,5.032533,78.469070,False,1.000292,1.003803,118.248221
3321,2020-01-08,89.52,90.0600,88.87,89.15,7696719,76.519868,76.981449,75.964261,76.203600,...,0.000953,0.000091,0.004694,0.370549,4.926150,78.469070,False,1.000296,1.003853,118.248221
3322,2020-01-09,90.21,90.2300,89.05,90.01,7517273,77.109666,77.126762,76.118122,76.938710,...,0.000953,0.000091,0.004594,0.370127,4.821328,78.469070,False,1.000294,1.003828,118.248221
3323,2020-01-10,89.06,90.2900,88.83,90.23,8581444,76.126669,77.178048,75.930070,77.126762,...,0.000953,0.000091,0.004485,0.370496,4.706619,78.469070,False,1.000292,1.003803,118.248221
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
477838,2022-12-19,81.56,84.9600,80.74,84.83,3342680,81.560000,84.960000,80.740000,84.830000,...,0.003694,0.001925,0.001303,0.338432,0.352850,86.239527,True,1.000873,1.011403,109.327511
477839,2022-12-20,82.57,82.7500,80.75,81.34,1548227,82.570000,82.750000,80.750000,81.340000,...,0.003694,0.001918,0.001339,0.337231,0.362455,86.239527,True,1.000873,1.011403,109.327511
477840,2022-12-21,82.60,84.8008,82.40,83.25,1668158,82.600000,84.800800,82.400000,83.250000,...,0.003694,0.001910,0.001365,0.335758,0.369449,86.239527,True,1.000865,1.011307,109.327511
477841,2022-12-22,81.24,82.8300,80.11,82.05,4667969,81.240000,82.830000,80.110000,82.050000,...,0.003694,0.001900,0.001388,0.334015,0.375844,86.239527,True,1.000873,1.011403,109.327511


In [54]:
sim["projected_return"] = (sim["prediction"] - sim["adjclose"]) / sim["adjclose"]
sim["projected_quarterly_return"] = (sim["quarterly_prediction"] - sim["adjclose"]) / sim["adjclose"]
sim["delta"] = [abs(x) for x in sim["projected_return"]]
sim["quarterly_delta"] = [abs(x) for x in sim["projected_quarterly_return"]]
sim["delta_sign"] = [1 if x >= 0 else -1 for x in sim["projected_return"]]
sim["quarterly_delta_sign"] = [1 if x >= 0 else -1 for x in sim["projected_quarterly_return"]]
sim["market_return"] = math.exp(np.log(1.15)/52)
sim["market_quarterly_return"] = math.exp(np.log(1.15)/4)
sim["rrr"] = sim["weekly_yield"] + sim["beta"] * (sim["market_return"] - sim["weekly_yield"]) - 1
sim["quarterly_rrr"] = sim["quarterly_yield"] + sim["quarterly_beta"] * (sim["market_quarterly_return"] - sim["quarterly_yield"]) - 1
sim = sim.groupby(["date","ticker"]).mean().reset_index()

In [55]:
sim.sort_values("date",inplace=True)
sim[["date","ticker","return_4","delta","quarterly_delta"
         ,"projected_quarterly_return","projected_return","quarterly_delta_sign","delta_sign","rrr","quarterly_rrr"]]

,date,ticker,return_4,delta,quarterly_delta,projected_quarterly_return,projected_return,quarterly_delta_sign,delta_sign,rrr,quarterly_rrr
0,2019-02-14,WAB,-0.017886,0.001510,5.921638,5.921638,-0.001510,1.0,-1.0,-0.000676,0.006232
1,2019-02-15,WAB,0.000000,0.039759,5.656487,5.656487,-0.039759,1.0,-1.0,-0.000690,0.006650
2,2019-02-25,WAB,-0.020822,0.044531,5.388727,5.388727,-0.044531,1.0,-1.0,-0.000530,0.006753
3,2019-02-26,WAB,-0.022045,0.008436,5.742888,5.742888,0.008436,1.0,1.0,-0.000222,0.006718
4,2019-02-27,WAB,-0.002867,0.009510,5.622895,5.622895,-0.009510,1.0,-1.0,0.000079,0.006727
...,...,...,...,...,...,...,...,...,...,...,...
61059,2022-12-23,DVN,-0.027356,0.011622,0.461305,-0.461305,0.011622,-1.0,1.0,0.000588,-0.003782
61058,2022-12-23,DIS,0.005905,0.071987,0.850957,0.850957,0.071987,1.0,1.0,0.000853,0.005208
61057,2022-12-23,DG,-0.001497,0.004888,0.230064,-0.230064,0.004888,-1.0,1.0,0.000497,0.016858
61063,2022-12-23,FAST,-0.003580,0.047627,0.058639,0.058639,0.047627,1.0,1.0,0.001101,0.025537


In [56]:
experimental.connect()
experimental.drop("complete")
experimental.store("complete",sim)
experimental.disconnect()

In [57]:
experimental.connect()
sim = experimental.retrieve("complete")
experimental.disconnect()

In [58]:
sim = sim.groupby(["date","ticker"]).mean().reset_index()

In [59]:
sim

,date,ticker,close,high,low,open,volume,adjclose,adjhigh,adjlow,...,projected_return,projected_quarterly_return,delta,quarterly_delta,delta_sign,quarterly_delta_sign,market_return,market_quarterly_return,rrr,quarterly_rrr
0,2019-02-14,WAB,72.05,73.4700,71.690,73.17,738.0,70.230026,71.614157,69.879120,...,-0.001510,5.921638,0.001510,5.921638,-1.0,1.0,1.002691,1.035558,-0.000676,0.006232
1,2019-02-15,WAB,74.92,75.0000,72.300,72.30,8.0,73.027531,73.105510,70.473711,...,-0.039759,5.656487,0.039759,5.656487,-1.0,1.0,1.002691,1.035558,-0.000690,0.006650
2,2019-02-25,WAB,78.06,80.4600,73.450,73.45,2469.0,76.088215,78.427591,71.594663,...,-0.044531,5.388727,0.044531,5.388727,-1.0,1.0,1.002691,1.035558,-0.000530,0.006753
3,2019-02-26,WAB,73.96,77.5700,73.200,76.33,27636580.0,72.091780,75.610592,71.350977,...,0.008436,5.742888,0.008436,5.742888,1.0,1.0,1.002691,1.035558,-0.000222,0.006718
4,2019-02-27,WAB,75.30,76.2100,72.230,73.47,5233075.0,73.397932,74.284945,70.405480,...,-0.009510,5.622895,0.009510,5.622895,-1.0,1.0,1.002691,1.035558,0.000079,0.006727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
61102,2022-12-23,WHR,141.62,141.6800,139.210,139.35,299475.0,139.839832,139.899078,137.460126,...,0.037983,2.315886,0.037983,2.315886,1.0,1.0,1.002691,1.035558,0.001177,0.025727
61103,2022-12-23,WRB,73.45,73.5600,72.775,73.06,642734.0,72.838674,72.947758,72.169292,...,0.000462,0.344646,0.000462,0.344646,1.0,1.0,1.002691,1.035558,0.000839,0.026275
61104,2022-12-23,WRK,35.46,35.4600,34.670,34.85,865191.0,35.163737,35.163737,34.380337,...,0.012505,0.795136,0.012505,0.795136,1.0,1.0,1.002691,1.035558,0.001112,0.042890
61105,2022-12-23,WYNN,80.72,81.3541,79.630,80.71,1134754.0,80.720000,81.354100,79.630000,...,0.068379,0.354404,0.068379,0.354404,1.0,1.0,1.002691,1.035558,0.001479,0.020724


In [60]:
reqs = [0.05]
signals = [0.05]
values = [True]
classifications = [False]
ceilings = [True,False]
floors = [True,False]
hedges = [False]
parameters = []
training_year = 4
for value in values:
    for classification in classifications:
        for ceiling in ceilings:
            for floor in floors:
                for hedge in hedges:
                    for signal in signals:
                        for req in reqs:
                            parameter = {"value":value
                                         ,"classification":classification
                                         ,"ceiling":ceiling
                                         ,"floor":floor
                                         ,"hedge":hedge
                                         ,"signal":signal
                                         ,"req":req
                                         ,"training_years":training_year}
                            parameters.append(parameter)

In [61]:
len(parameters)

4

In [62]:
parameter = parameters[0]

In [63]:
experimental.connect()
experimental.drop("trades")
for parameter in tqdm(parameters):
    current_sim = sim.copy()
    b.experimental_backtest(current_sim,parameter,start_date,end_date,experimental)
experimental.disconnect()

100%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 4/4 [00:07<00:00,  1.80s/it]
